<a href="https://colab.research.google.com/github/preetham-p-m/llm/blob/main/minutes_of_the_meeting/meeting_minutes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate genai

In [ ]:
import os
import requests
from IPython.display import Markdown, display, update_display
from google import genai
from google.genai import types
from google.colab import userdata, drive
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [ ]:
AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Llama-3.1-8B-Instruct"

In [ ]:
drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/llm_helpers/meeting.mp3"

In [ ]:
hf_token = userdata.get("HUGGINGFACE_API_KEY")
login(hf_token)

In [ ]:
gemini_token = userdata.get("GEMINI_API_KEY")
client = genai.Client(api_key=gemini_token)

In [ ]:
with open(audio_filename, 'rb') as f:
    audio_bytes = f.read()

response = client.models.generate_content(
  model='gemini-2.0-flash',
  contents=[
    'Describe this audio clip',
    types.Part.from_bytes(
      data=audio_bytes,
      mime_type='audio/mp3',
    )
  ]
)

transcription = response.candidates[0].content.parts[0].text

In [ ]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

In [ ]:
response = tokenizer.decode(outputs[0])
response

In [ ]:
display(Markdown(response))

# Output

**Minutes of the Special Meeting of the Alameda City Council**
===========================================================

### Summary
#### Attendees:
- Mayor
- City Clerk
- Members of the Alameda City Council
#### Location:
- Virtual meeting via Zoom
#### Date:
- [Insert Date]

**Summary:** The special meeting of the Alameda City Council was held via Zoom to discuss amendments to the City Charter, specifically addressing potential measures related to Council interference. The meeting aimed to provide direction on the proposed amendments, emphasizing the importance of aligning the City Charter with current needs and realities.

### Key Discussion Points
* Overview of the meeting process and public comment submission methods
* Importance of updating the City Charter to reflect current realities and needs
* Acknowledgement of past turbulent events and commitment to restoring public trust
* Discussion on providing direction on potential measures amending the city charter

### Takeaways
* The City Charter requires updates to align with current needs and realities.
* The Council is committed to restoring public trust and addressing governance issues.
* Utilizing technology, such as Zoom, enables the Council to maintain accessibility and facilitate public participation.

### Action Items
* **Owner: City Clerk**
  - Finalize the proposed amendments to the City Charter for review and discussion by the Council.
* **Owner: Mayor**
  - Provide a detailed report on the current state of the City Charter and its alignment with current needs and realities.
* **Owner: Council Members**
  - Review and provide feedback on the proposed amendments to the City Charter.
  - Collaborate with the City Clerk to ensure a smooth and accessible meeting process.